In [216]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np

### Defining Setup Time and Hourly Yield Constants

In [217]:
T = pd.read_csv('setup_constants.csv')
R = pd.read_csv('yield_constants.csv')

In [218]:
T.index += 1
R.index += 1

### Defining Weekly Demand Constants

In [219]:
D = pd.read_csv('weekly_demands.csv')

In [220]:
D.index += 1

# Defining Problem

In [221]:
model = gp.Model('base_problem')

## Variables

### Adding Binary Setup Variables

In [ ]:
Sw = []
for w in range(2):
    S = []
    for i in range(5):
        S_i = []
        for j in range(5):
            S_i.append(model.addVar(vtype=GRB.BINARY, name=f"S_{i}_{j}_{w}", lb=0))
        S.append(S_i)
    Sw.append(S)

### Adding Binary Overtime Setup Variables

In [224]:
Spw = []
for w in range(2):
    Sp = []
    for i in range(5):
        Sp_i = []
        for j in range(5):
            Sp_i.append(model.addVar(vtype=GRB.BINARY, name=f"Sp_{i}_{j}_{w}", lb=0))
        Sp.append(Sp_i)
    Spw.append(Sp)

### Adding Production Variables

In [225]:
Pw = []
for w in range(2):
    P = []
    for i in range(5):
        P_i = []
        for j in range(5):
            P_i.append(model.addVar(vtype=GRB.CONTINUOUS, name=f"P_{i}_{j}_{w}", lb=0))
        P.append(P_i)
    Pw.append(P)

### Adding Overtime Production Variables

In [226]:
Ppw = []
for w in range(2):
    Pp = []
    for i in range(5):
        Pp_i = []
        for j in range(5):
            Pp_i.append(model.addVar(vtype=GRB.CONTINUOUS, name=f"Pp_{i}_{j}_{w}", lb=0))
        Pp.append(Pp_i)
    Ppw.append(Pp)

### Adding Use Variables

In [227]:
Uw = []
for w in range(2):
    U = []
    for i in range(5):
        U_i = []
        for j in range(5):
            U_i.append(model.addVar(vtype=GRB.BINARY, name=f"U_{i}_{j}_{w}", lb=0))
        U.append(U_i)
    Uw.append(U)

### Adding Overtime Use Variables

In [228]:
Upw = []
for w in range(2):
    Up = []
    for i in range(5):
        Up_i = []
        for j in range(5):
            Up_i.append(model.addVar(vtype=GRB.BINARY, name=f"Up_{i}_{j}_{w}", lb=0))
        Up.append(U_i)
    Upw.append(Up)

## Constraints

### Demand Constraints

In [229]:
for w in range(2):
    for i in range(5):
        constr_expr = gp.quicksum(Pw[w][j][i]*R.iloc[j,i] + Ppw[w][j][i]*R.iloc[j,i] for j in range(5))
        model.addConstr(constr_expr >= D.iloc[w][i], name=f'Demand_Prod_{i+1}_{w}')

/var/folders/w_/z02fg3l56_jfc9tys40hqtdm0000gn/T/ipykernel_22872/1960682158.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  model.addConstr(constr_expr >= D.iloc[w][i], name=f'Demand_Prod_{i+1}_{w}')


### Normal Production Time Constraints

In [230]:
for w in range(2):
    for i in range(5):
        constr_expr = gp.quicksum(Sw[w][i][j]*T.iloc[i,j] + Pw[w][i][j] for j in range(5))
        model.addConstr(constr_expr <= 120, name=f'Prod_Hrs_Mach_{i+1}_{w}')

### Overtime Production Time Constraints

In [231]:
for w in range(2):
    for i in range(5):
        constr_expr = gp.quicksum(Spw[w][i][j]*T.iloc[i,j] + Ppw[w][i][j] for j in range(5))
        model.addConstr(constr_expr <= 48, name=f'OT_Prod_Hrs_Mach_{i+1}_{w}')

### Production Use Linkage Constants
- Regular Time and Overtime are both included here

In [232]:
for w in range(2):
    for i in range(5):
        for j in range(5):
            model.addConstr(Pw[w][i][j]*R.iloc[i,j] <= 10000*Uw[w][i][j], name=f'UseProdLinkage_Mach_{i+1}_Prod_{j+1}_Week_{w}')
            model.addConstr(Ppw[w][i][j]*R.iloc[i,j] <= 10000*(Uw[w][i][j]+Upw[w][i][j]), name=f'OTUseProdLinkage_Mach_{i+1}_Prod_{j+1}_Week_{w}')

### Setup Use Linkage Constants
- Regular Time and Overtime are both included here

In [233]:
for i in range(5):
    for j in range(5):
        model.addConstr(Uw[0][i][j] <= 10*Sw[0][i][j], name=f'SetupUseLinkage_Mach_{i+1}_Prod_{j+1}_Week_0')
        model.addConstr(Upw[0][i][j] <= 10*(Sw[0][i][j]+Spw[0][i][j]), name=f'SetupUseLinkage_Mach_{i+1}_Prod_{j+1}_Week_0')

for i in range(5):
    for j in range(5):
        model.addConstr(Uw[1][i][j] <= 10*(Sw[1][i][j]+Sw[0][i][j]+Spw[0][i][j]), name=f'SetupUseLinkage_Mach_{i+1}_Prod_{j+1}_Week_1')
        model.addConstr(Upw[1][i][j] <= 10*(Sw[0][i][j]+Spw[0][i][j]+Sw[1][i][j]+Spw[1][i][j]), name=f'SetupUseLinkage_Mach_{i+1}_Prod_{j+1}_Week_1')

## Objective Function

In [234]:
obj_parts = []
for w in range(2):
    for i in range(5):
        obj_parts.append(gp.quicksum(Spw[w][i][j]*T.iloc[i,j] + Ppw[w][i][j] for j in range(5)))

In [235]:
model.setObjective(gp.quicksum(obj_parts),GRB.MINIMIZE)

In [236]:
model.update()

In [237]:
model.getObjective()

<gurobi.LinExpr: 8.0 Sp_0_0_0 + 8.0 Sp_0_3_0 + 10.0 Sp_1_0_0 + 8.0 Sp_1_1_0 + 10.0 Sp_2_1_0 + 24.0 Sp_2_4_0 + 8.0 Sp_3_1_0 + 12.0 Sp_3_2_0 + 8.0 Sp_4_3_0 + 20.0 Sp_4_4_0 + 8.0 Sp_0_0_1 + 8.0 Sp_0_3_1 + 10.0 Sp_1_0_1 + 8.0 Sp_1_1_1 + 10.0 Sp_2_1_1 + 24.0 Sp_2_4_1 + 8.0 Sp_3_1_1 + 12.0 Sp_3_2_1 + 8.0 Sp_4_3_1 + 20.0 Sp_4_4_1 + Pp_0_0_0 + Pp_0_1_0 + Pp_0_2_0 + Pp_0_3_0 + Pp_0_4_0 + Pp_1_0_0 + Pp_1_1_0 + Pp_1_2_0 + Pp_1_3_0 + Pp_1_4_0 + Pp_2_0_0 + Pp_2_1_0 + Pp_2_2_0 + Pp_2_3_0 + Pp_2_4_0 + Pp_3_0_0 + Pp_3_1_0 + Pp_3_2_0 + Pp_3_3_0 + Pp_3_4_0 + Pp_4_0_0 + Pp_4_1_0 + Pp_4_2_0 + Pp_4_3_0 + Pp_4_4_0 + Pp_0_0_1 + Pp_0_1_1 + Pp_0_2_1 + Pp_0_3_1 + Pp_0_4_1 + Pp_1_0_1 + Pp_1_1_1 + Pp_1_2_1 + Pp_1_3_1 + Pp_1_4_1 + Pp_2_0_1 + Pp_2_1_1 + Pp_2_2_1 + Pp_2_3_1 + Pp_2_4_1 + Pp_3_0_1 + Pp_3_1_1 + Pp_3_2_1 + Pp_3_3_1 + Pp_3_4_1 + Pp_4_0_1 + Pp_4_1_1 + Pp_4_2_1 + Pp_4_3_1 + Pp_4_4_1>

In [238]:
model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.0.0 24A348)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 230 rows, 300 columns and 715 nonzeros
Model fingerprint: 0xcfe04c60
Variable types: 100 continuous, 200 integer (200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 4e+03]
Found heuristic solution: objective 328.1214976
Presolve removed 165 rows and 238 columns
Presolve time: 0.00s
Presolved: 65 rows, 62 columns, 198 nonzeros
Variable types: 34 continuous, 28 integer (28 binary)

Root relaxation: objective 1.522458e+02, 56 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  152.24584    0    9  328.12150  152.24

In [239]:
model.Status == GRB.OPTIMAL

True

In [240]:
for i in range(5):
    print(f'Machine {i+1}: ', end='')
    for j in range(5):
        print(int(abs(Sw[0][i][j].X*T.iloc[i,j])), end=" ")
    print("")

Machine 1: 8 0 0 8 0 
Machine 2: 10 0 0 0 0 
Machine 3: 0 10 0 0 24 
Machine 4: 0 8 12 0 0 
Machine 5: 0 0 0 8 20 


In [241]:
for i in range(5):
    print(f'Machine {i+1}: ', end='')
    for j in range(5):
        print(int(abs(Sw[1][i][j].X*T.iloc[i,j])), end=" ")
    print("")

Machine 1: 0 0 0 0 0 
Machine 2: 0 8 0 0 0 
Machine 3: 0 0 0 0 0 
Machine 4: 0 0 0 0 0 
Machine 5: 0 0 0 0 0 


In [242]:
for i in range(5):
    print(f'Machine {i+1}: ', end='')
    for j in range(5):
        print(int(abs(Spw[0][i][j].X*T.iloc[i,j])), end=" ")
    print("")

Machine 1: 0 0 0 0 0 
Machine 2: 0 0 0 0 0 
Machine 3: 0 0 0 0 0 
Machine 4: 0 0 0 0 0 
Machine 5: 0 0 0 0 0 


In [243]:
for i in range(5):
    print(f'Machine {i+1}: ', end='')
    for j in range(5):
        print(int(abs(Spw[1][i][j].X*T.iloc[i,j])), end=" ")
    print("")

Machine 1: 0 0 0 0 0 
Machine 2: 0 0 0 0 0 
Machine 3: 0 0 0 0 0 
Machine 4: 0 0 0 0 0 
Machine 5: 0 0 0 0 0 


In [244]:
for i in range(5):
    print(f'Machine {i+1}: ', end='')
    for j in range(5):
        print((abs(Pw[0][i][j].X)), end=" ")
    print("")

Machine 1: 49.583333333333336 0.0 0.0 54.416666666666664 0.0 
Machine 2: 110.0 0.0 0.0 0.0 0.0 
Machine 3: 0.0 86.0 0.0 0.0 0.0 
Machine 4: 0.0 0.0 100.0 0.0 0.0 
Machine 5: 0.0 0.0 0.0 46.666666666666664 45.333333333333336 


In [245]:
for i in range(5):
    print(f'Machine {i+1}: ', end='')
    for j in range(5):
        print((abs(Pw[1][i][j].X)), end=" ")
    print("")

Machine 1: 36.41025641025641 0.0 0.0 83.58974358974359 0.0 
Machine 2: 101.24542124542124 10.75457875457876 0.0 0.0 0.0 
Machine 3: 0.0 120.0 0.0 0.0 0.0 
Machine 4: 0.0 13.333333333333334 106.66666666666667 0.0 0.0 
Machine 5: 0.0 0.0 0.0 26.666666666666668 93.33333333333333 


In [246]:
for i in range(5):
    print(f'Machine {i+1}: ', end='')
    for j in range(5):
        print((abs(Ppw[0][i][j].X)), end=" ")
    print("")

Machine 1: 0.0 0.0 0.0 1.480769230769233 0.0 
Machine 2: 0.0 0.0 0.0 0.0 0.0 
Machine 3: 0.0 47.5959595959596 0.0 0.0 0.0 
Machine 4: 0.0 41.333333333333336 6.666666666666667 0.0 0.0 
Machine 5: 0.0 0.0 0.0 0.0 48.0 


In [247]:
for i in range(5):
    print(f'Machine {i+1}: ', end='')
    for j in range(5):
        print((abs(Ppw[1][i][j].X)), end=" ")
    print("")

Machine 1: 0.0 0.0 0.0 0.0 0.0 
Machine 2: 0.0 0.0 0.0 0.0 0.0 
Machine 3: 0.0 0.0 0.0 0.0 0.0 
Machine 4: 0.0 21.58222729651301 0.0 0.0 0.0 
Machine 5: 0.0 0.0 0.0 0.0 0.0 


In [248]:
print("**** Week 1 ****")
for i in range(5):
    print(f"Part {i+1}:", round(sum([abs(Pw[0][j][i].X+Ppw[0][j][i].X)*R.iloc[j,i] for j in range(5)])), end=" ")
    print("")

**** Week 1 ****
Part 1: 3500 
Part 2: 3000 
Part 3: 4000 
Part 4: 4000 
Part 5: 2800 


In [249]:
print("**** Week 2 ****")
for i in range(5):
    print(f"Part {i+1}:", round(sum([abs(Pw[1][j][i].X+Ppw[1][j][i].X)*R.iloc[j,i] for j in range(5)])), end=" ")
    print("")

**** Week 2 ****
Part 1: 3000 
Part 2: 2800 
Part 3: 4000 
Part 4: 4300 
Part 5: 2800 


In [250]:
print('Final Objective Value: ', model.ObjVal)

Final Objective Value:  166.65895612324186
